# Factor strategy

In [88]:
import os
import sys

# 현재 작업 디렉토리와 프로젝트 루트 확인
current_dir = os.getcwd()  # /Users/jh/Desktop/PyRA/output
project_root = os.path.dirname(current_dir)  # /Users/jh/Desktop/PyRA

# data 폴더의 절대 경로를 파이썬 경로에 추가
data_path = os.path.join(project_root, 'data')
sys.path.insert(0, data_path)
sys.path.insert(0, project_root)

print("Python paths added:")
print("1.", project_root)
print("2.", data_path)

from typing import Dict, Optional
import pandas as pd
import plotly.express as px

# 직접 data_loader 임포트 시도
try:
    from data_loader import PykrxDataLoader
    print("Successfully imported PykrxDataLoader")
except ImportError as e:
    print("Import error:", e)

from simulation.account import Account
from simulation.broker import Broker
from simulation.metric import cagr, mdd, sharpe_ratio, sortino_ratio
from simulation.utility import get_lookback_fromdate, rebalance

Python paths added:
1. /Users/jh/Desktop
2. /Users/jh/Desktop/data
Successfully imported PykrxDataLoader


In [25]:
def calculate_momentum(ohlcv_data: pd.DataFrame,
                       lookback_period: int,
                       skip_period: int) -> pd.DataFrame:
    # 데이터 재구조화
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['close']

    # 팩터 계산(모멘텀)
    momentum_data = data.shift(periods=skip_period).rolling(
        window=lookback_period).apply(lambda x: x[-1] / x[0] - 1)

    return momentum_data

In [26]:
def calculate_fundamental(ohlcv_data: pd.DataFrame,
                          market_cap_data: pd.DataFrame,
                          fundamental_data: pd.DataFrame,
                          strategy_name: str,
                          lookback_period: int = 12, ) -> pd.DataFrame:
    # 룩백 길이 변환
    if ohlcv_data.frequency == 'd': lookback_period = int(lookback_period / 21)

    # 데이터 재구조화(OHLCV)
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['close']

    # 데이터 조정 및 재구조화(기본)
    fundamental_data = fundamental_adjuster(fundamental_data=fundamental_data,
                                            market_cap_data=market_cap_data)
    mapping = {'per': 'EPS', 'pbr': 'BPS', 'dividend': 'DPS'}
    target_fundamental = mapping.get(strategy_name)
    fundamental_data = fundamental_data[
        ['date', target_fundamental, 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)[target_fundamental]

    # 펀더멘탈 데이터 날짜 수정
    fundamental_data = date_adjust(index_df=data, df=fundamental_data)

    # 팩터 계산(PER, PBR, 배당)
    if strategy_name == "per":
        fundamental_data = fundamental_data.rolling(
            window=lookback_period).sum()
        fundamental_data = data / fundamental_data
    elif strategy_name == 'pbr':
        fundamental_data = data / fundamental_data
    elif strategy_name == 'dividend':
        fundamental_data = fundamental_data / data
    else:
        raise ValueError

    return fundamental_data


def fundamental_adjuster(fundamental_data: pd.DataFrame,
                         market_cap_data: pd.DataFrame) -> pd.DataFrame:
    # 현재 주식 수로 나누기
    market_cap_data['shares_div'] = market_cap_data['shares'].div(
        market_cap_data.groupby('ticker')['shares'].transform('last'))
    market_cap_data = market_cap_data[['shares_div', 'ticker']].reset_index()

    # 조정(액면분할 반영)
    data = fundamental_data.reset_index().merge(
        market_cap_data, on=['date', 'ticker']).set_index(['date'])
    data['EPS'] = data['shares_div'] * data['EPS']
    data['BPS'] = data['shares_div'] * data['BPS']
    data['DPS'] = data['shares_div'] * data['DPS']

    # 주식 수 바뀐 당일 오류 탐색
    changed_row = data.copy()
    changed_row['previous'] = changed_row.shares_div.shift(1) #1개월 뒤 공시, 데이터 날짜 교정(주식 데이터와 펀더멘탈 데이터 시차 교정)
    changed_row = changed_row[changed_row.shares_div != changed_row.previous]
    changed_row = changed_row.reset_index().set_index(['ticker', 'date'])

    # 당일 오류 정정
    data = data.reset_index().set_index(['ticker', 'date'])
    for i, index in enumerate(data.index):
        if index in changed_row.index:
            data.loc[index] = data.iloc[i + 1]

    return data.reset_index()[['ticker', 'date', 'BPS', 'EPS', 'DPS']]

In [27]:
#소형주 전략
def calculate_small(ohlcv_data: pd.DataFrame,
                    market_cap_data: pd.DataFrame) -> pd.DataFrame:
    # 데이터 재구조화(OHLCV)
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['close']

    # 데이터 재구조화(주식 수)
    market_cap_data = market_cap_data[['shares',
                                       'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['shares']

    # 시장 데이터 날짜 수정
    market_cap_data = date_adjust(index_df=data, df=market_cap_data)

    # 팩터 계산(시가총액)
    market_cap_data = market_cap_data * data

    return market_cap_data

In [28]:
#로우볼 전략
def calculate_lowvol(ohlcv_data: pd.DataFrame,
                     lookback_period: int) -> pd.DataFrame:
    # 데이터 재구조화
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['close']

    # 팩터 계산(표준편차)
    std_data = data.pct_change().rolling(lookback_period).std()

    return std_data

In [29]:
#수급주체 전략
def calculate_trader(ohlcv_data: pd.DataFrame,
                     market_cap_data: pd.DataFrame,
                     trader_data: pd.DataFrame,
                     strategy_name: str,
                     lookback_period: Optional[int] = 1) -> pd.DataFrame:
    # 룩백 길이 변환
    if ohlcv_data.frequency == 'd': lookback_period = int(lookback_period / 21)

    # 데이터 재구조화(OHLCV)
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['close']

    # 데이터 재구조화(주식 수)
    market_cap_data = market_cap_data[['shares',
                                       'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['shares']

    # 데이터 재구조화(수급 주체)
    trader_data = trader_data[[strategy_name,
                               'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)[strategy_name]
    trader_data = trader_data.rolling(window=lookback_period).sum()

    # 시장 및 수급 주체 데이터 날짜 수정
    market_cap_data = date_adjust(index_df=data, df=market_cap_data)
    trader_data = date_adjust(index_df=data, df=trader_data)

    # 팩터 계산(수급 주체)
    market_cap_data = market_cap_data * data
    trader_data = trader_data / market_cap_data

    return trader_data

In [59]:
def downsample_df(df: pd.DataFrame) -> pd.DataFrame:
    data = df.copy()
    data.index = pd.to_datetime(data.index)
    data['date'] = data.index
    data = data.resample('M').apply('last')
    return data.set_index(pd.DatetimeIndex(data.date)).drop(columns=['date'])

In [83]:
#멀티팩터 전략
def calculate_multifactor(ohlcv_data: pd.DataFrame,
                          oracle: bool = False) -> pd.DataFrame:
    # 포트폴리오 csv 가져오기
    filename = 'factor/ticker_weight_real.csv' if oracle else 'factor/ticker_weight_pred.csv'

    # 데이터 읽기 및 전처리
    premade = pd.read_csv(filename)
    
    # 데이터프레임으로 직접 변환
    dates = sorted(premade['date'].unique())
    tickers = sorted(premade['ticker'].astype(str).str.zfill(6).unique())
    
    # 빈 데이터프레임 생성 (float 타입으로)
    result = pd.DataFrame(0.0, 
                         index=pd.to_datetime(dates), 
                         columns=tickers)
    
    # 값 채우기
    for _, row in premade.iterrows():
        date = pd.to_datetime(row['date'])
        ticker = str(row['ticker']).zfill(6)
        result.at[date, ticker] = float(row['weight'])
    
    # 데이터 크기 맞추기
    date_pad = downsample_df(ohlcv_data).drop(columns=ohlcv_data.columns)
    padded_premade = pd.concat([date_pad, result])
    padded_premade = padded_premade[
        ~padded_premade.index.duplicated(keep='last')]

    return padded_premade.sort_index()

In [85]:
#가중치 산정 함수 정의
def get_factor_weight(factor_data: pd.DataFrame,
                      buying_ratio: float,
                      strategy_name: str) -> Optional[Dict]:
    # 데이터 중 결측치가 있는지 확인함
    if factor_data.isnull().values.any():
        return None

    # 매수 주식 선정
    reverse = {'per', 'pbr', 'small', 'lovwol'} # PBR,PER,소형주,로우볼 전략 선택
    ratio = buying_ratio if strategy_name in reverse else 1 - buying_ratio #분위 개념
    top_quantile = factor_data.quantile(ratio)
    if strategy_name in reverse:
        stocks_to_buy = factor_data[factor_data <= top_quantile].index.to_list()
    else:
        stocks_to_buy = factor_data[factor_data >= top_quantile].index.to_list()

    # 주식 비율 할당
    weights = 1 / len(stocks_to_buy) if stocks_to_buy else 0
    portfolio = {ticker: weights if ticker in stocks_to_buy else 0.0 for ticker
                 in factor_data.index}

    return portfolio

In [53]:
#시뮬레이션 함수
def simulate_factor(ohlcv_data: pd.DataFrame,
                    market_cap_data: Optional[pd.DataFrame],
                    fundamental_data: Optional[pd.DataFrame],
                    trader_data: Optional[pd.DataFrame],
                    lookback_period: Optional[int],
                    skip_period: Optional[int],
                    strategy_name: str,
                    buying_ratio: float = 0.1) -> Account:
    # 계좌 및 브로커 선언
    account = Account(initial_cash=100000000)
    broker = Broker()

    # 팩터 계산(상대모멘텀, 펀데멘탈(pbr,per,배당), 소형주,,, 등 전략별 팩터 계수(베타) 분석)
    if strategy_name == 'relative':
        factor_data = calculate_momentum(ohlcv_data=ohlcv_data,
                                         lookback_period=lookback_period,
                                         skip_period=skip_period, )
    elif strategy_name in {'per', 'pbr', 'dividend'}:
        factor_data = calculate_fundamental(ohlcv_data=ohlcv_data,
                                            market_cap_data=market_cap_data,
                                            fundamental_data=fundamental_data,
                                            lookback_period=lookback_period,
                                            strategy_name=strategy_name)
    elif strategy_name == 'small':
        factor_data = calculate_small(ohlcv_data=ohlcv_data,
                                      market_cap_data=market_cap_data)
    elif strategy_name in {'individual', 'institutional', 'foreign'}:
        factor_data = calculate_trader(ohlcv_data=ohlcv_data,
                                       market_cap_data=market_cap_data,
                                       trader_data=trader_data,
                                       lookback_period=lookback_period,
                                       strategy_name=strategy_name)
    elif strategy_name == 'lowvol':
        factor_data = calculate_lowvol(ohlcv_data=ohlcv_data,
                                       lookback_period=lookback_period)
    elif strategy_name == 'multifactor':
        factor_data = calculate_multifactor(ohlcv_data=ohlcv_data)
    else:
        raise ValueError
    # 월별 리밸런싱 날짜 추출
    month_end = downsample_df(ohlcv_data).index

    for date, ohlcv in ohlcv_data.groupby(['date']):
        date=pd.to_datetime(date[0])
        print(date.date())
        # 주문 집행 및 계좌 갱신
        transactions = broker.process_order(dt=date, data=ohlcv,
                                            orders=account.orders)
        account.update_position(transactions=transactions)
        account.update_portfolio(dt=date, data=ohlcv)
        account.update_order()

        # 리밸런싱 날짜가 아닐 경우 넘어가기
        if date not in month_end:
            continue

        # 팩터 전략을 이용하여 포트폴리오 구성
        factor_data_slice = factor_data.loc[date]
        weights = get_factor_weight(factor_data=factor_data_slice,
                                    buying_ratio=buying_ratio,
                                    strategy_name=strategy_name)

        print(f'Portfolio: {weights}')
        if weights is None:
            continue

        # 포트폴리오 비율 갱신
        account.update_weight(dt=date, weight=weights)

        # 주문 생성
        rebalance(dt=date, data=ohlcv, account=account, weights=weights)

    # 리밸런싱 날짜만 남기기
    account.account_history = [item for item in account.account_history if
                               item['date'].date() in month_end.date]

    return account

In [7]:
# 데이터 시작과 끝 날짜 정의
fromdate = '2020-01-01'
todate = '2025-01-11'

# 투자할 종목 후보 정의
ticker_list = ['000660', '005490', '051910', '006400', '005380', '000270',
               '012330', '068270', '105560', '096770', '055550', '066570',
               '047050', '032830', '015760', '086790', '000810', '033780',
               '034730', '034020', '009150', '138040', '010130', '001570',
               '010950', '024110', '030200', '051900', '009830', '086280',
               '011170', '011070', '012450', '036570', '005830', '161390',
               '034220', '004020', '032640', '097950', '000720', '006800',
               '006260', '010620', '011780', '078930', '005940', '029780',
               '128940', '035250', '016360', '021240', '010120', '052690',
               '008770', '071050', '000990', '001450', '020150', '039490',
               '111770', '000880', '004370', '036460', '007070', '138930',
               '139480', ]

class FactorData:
    def __init__(self, fromdate, todate, ticker_list):
        self.data_loader = PykrxDataLoader(fromdate=fromdate, todate=todate, market="KOSPI")
        
        # 기본 데이터 로드
        self.ohlcv_data = self.data_loader.load_stock_data(
            ticker_list=ticker_list, freq='m', delay=1)
        self.ohlcv_data_day = self.data_loader.load_stock_data(
            ticker_list=ticker_list, freq='d', delay=1)
        self.fundamental_data = self.data_loader.load_fundamental_data(
            ticker_list=ticker_list, freq='m', delay=1)
        self.market_cap_data = self.data_loader.load_market_cap_data(
            ticker_list=ticker_list, freq='m', delay=1)
        self.trader_data = self.data_loader.load_trader_data(
            ticker_list=ticker_list, freq='m', delay=1)
    
    def get_sliced_data(self, fromdate):
        return {
            'ohlcv_data': df_slicer(self.ohlcv_data_day, fromdate),
            'market_cap_data': df_slicer(self.market_cap_data, fromdate),
            'fundamental_data': df_slicer(self.fundamental_data, fromdate),
            'trader_data': df_slicer(self.trader_data, fromdate)
        }

# 데이터 초기화
factor_data = FactorData(fromdate=fromdate, todate=todate, ticker_list=ticker_list)

/Users/jh/Desktop/PyRA/data/data_loader.py:116: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.groupby('ticker').resample(freq).apply(rule).reset_index(level=0)
/Users/jh/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)
/Users/jh/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)
/Users/jh/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)
/Users/jh/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: Fut

In [10]:
#전략 실행
simulation_fromdate = '2020-01-01' # 시작일 지정

def df_slicer(df: pd.DataFrame, fromdate: str):
    frequency = df.frequency if hasattr(df, 'frequency') else None
    
    # 인덱스가 정렬되어 있지 않은 경우를 처리
    df = df.sort_index()
    
    # fromdate가 실제 존재하는 날짜가 되도록 조정
    available_dates = df.index.unique()
    closest_date = available_dates[available_dates >= fromdate][0]
    
    sliced_df = df[closest_date:]
    sliced_df.__setattr__('frequency', frequency)
    
    return sliced_df

In [35]:
lookback = 3
offset = 1
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                     lookback=lookback + offset, freq='m')
ratio = 0.1
strategy = 'relative'

sliced_data = factor_data.get_sliced_data(total_lookback)
account_relative = simulate_factor(
    ohlcv_data=sliced_data['ohlcv_data'],
    market_cap_data=None,
    fundamental_data=None,
    trader_data=None,
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/Users/jh/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/2994663790.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  window=lookback_period).apply(lambda x: x[-1] / x[0] - 1)
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')


2020-01-02
2020-01-03
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-28
2020-01-29
2020-01-30
2020-01-31
Portfolio: None
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
Portfolio: None
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-30
2020-03-31
Portfolio: None
2020-04-01
2020-04-02
2020-04-03
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-13
2020-04-14
2020-04-16
2020-04-17
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-27
2020-04-28
2020-04-29
Portfolio: None
2020-05-04
2020-05-06
2020-05-07
2

In [36]:
lookback = 12
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                     lookback=lookback + offset, freq='m')
ratio = 0.1
strategy = 'per'

sliced_data = factor_data.get_sliced_data(total_lookback)
account_per = simulate_factor(
    ohlcv_data=sliced_data['ohlcv_data'],
    market_cap_data=sliced_data['market_cap_data'],
    fundamental_data=sliced_data['fundamental_data'],
    trader_data=None,
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/Users/jh/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')


2020-01-02
2020-01-03
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-28
2020-01-29
2020-01-30
2020-01-31
Portfolio: None
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
Portfolio: None
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-30
2020-03-31
Portfolio: None
2020-04-01
2020-04-02
2020-04-03
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-13
2020-04-14
2020-04-16
2020-04-17
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-27
2020-04-28
2020-04-29
Portfolio: None
2020-05-04
2020-05-06
2020-05-07
2

In [37]:
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                     lookback=lookback + offset, freq='m')
ratio = 0.1
strategy = 'pbr'

sliced_data = factor_data.get_sliced_data(total_lookback)
account_pbr = simulate_factor(
    ohlcv_data=sliced_data['ohlcv_data'],
    market_cap_data=sliced_data['market_cap_data'],
    fundamental_data=sliced_data['fundamental_data'],
    trader_data=None,
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/Users/jh/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')


2020-01-02
2020-01-03
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-28
2020-01-29
2020-01-30
2020-01-31
Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.14285714285714285, '000990': 0.0, '001450': 0.0, '001570': 0.14285714285714285, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.14285714285714285, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.14285714285714285, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.0, '020150': 0.0, '021240': 0.0, '024110': 0.14285714285714285, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.14285714285714285, '034220': 0.0, '034730': 0.0, '035250': 0.142857142857142

In [38]:
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                     lookback=lookback + offset, freq='m')
ratio = 0.1
strategy = 'dividend'

sliced_data = factor_data.get_sliced_data(total_lookback)
account_dividend = simulate_factor(
    ohlcv_data=sliced_data['ohlcv_data'],
    market_cap_data=sliced_data['market_cap_data'],
    fundamental_data=sliced_data['fundamental_data'],
    trader_data=None,
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/Users/jh/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')


2020-01-02
2020-01-03
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-28
2020-01-29
2020-01-30
2020-01-31
Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.14285714285714285, '000990': 0.0, '001450': 0.0, '001570': 0.14285714285714285, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.14285714285714285, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.14285714285714285, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.14285714285714285, '015760': 0.0, '016360': 0.0, '020150': 0.0, '021240': 0.0, '024110': 0.14285714285714285, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.142857142857142

In [39]:
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                     lookback=lookback + offset, freq='m')
ratio = 0.1
strategy = 'small'

sliced_data = factor_data.get_sliced_data(total_lookback)
account_small = simulate_factor(
    ohlcv_data=sliced_data['ohlcv_data'],
    market_cap_data=sliced_data['market_cap_data'],
    fundamental_data=None,
    trader_data=None,
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/Users/jh/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')


2020-01-02
2020-01-03
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-28
2020-01-29
2020-01-30
2020-01-31
Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.14285714285714285, '001450': 0.0, '001570': 0.14285714285714285, '004020': 0.0, '004370': 0.14285714285714285, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.14285714285714285, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.0, '020150': 0.0, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.14285714285714285, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.0

In [40]:
lookback = 60
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                     lookback=lookback + offset, freq='d')
ratio = 0.1
strategy = 'lowvol'

sliced_data = factor_data.get_sliced_data(total_lookback)
account_lowvol = simulate_factor(
    ohlcv_data=sliced_data['ohlcv_data'],
    market_cap_data=None,
    fundamental_data=None,
    trader_data=None,
    lookback_period=lookback,
    skip_period=offset,
    strategy_name=strategy,
    buying_ratio=ratio)

/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')


2020-01-02
2020-01-03
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-28
2020-01-29
2020-01-30
2020-01-31
Portfolio: None
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
Portfolio: None
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-30
2020-03-31
Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.0, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008

In [41]:
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                     lookback=lookback + offset, freq='m')
ratio = 0.1
strategy = 'individual'

sliced_data = factor_data.get_sliced_data(total_lookback)
account_individual = simulate_factor(
    ohlcv_data=sliced_data['ohlcv_data'],
    market_cap_data=sliced_data['market_cap_data'],
    fundamental_data=None,
    trader_data=sliced_data['trader_data'],
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/Users/jh/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')


2020-01-02
2020-01-03
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-28
2020-01-29
2020-01-30
2020-01-31
Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.14285714285714285, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.0, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.14285714285714285, '006260': 0.14285714285714285, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.14285714285714285, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.0, '020150': 0.0, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.14285714285714285, '036570': 0.0

In [42]:
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                     lookback=lookback + offset, freq='m')
ratio = 0.1
strategy = 'institutional'

sliced_data = factor_data.get_sliced_data(total_lookback)
account_institutional = simulate_factor(
    ohlcv_data=sliced_data['ohlcv_data'],
    market_cap_data=sliced_data['market_cap_data'],
    fundamental_data=None,
    trader_data=sliced_data['trader_data'],
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)


/Users/jh/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')


2020-01-02
2020-01-03
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-28
2020-01-29
2020-01-30
2020-01-31
Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.0, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.0, '007070': 0.14285714285714285, '008770': 0.0, '009150': 0.0, '009830': 0.14285714285714285, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.14285714285714285, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.0, '020150': 0.0, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.14285714285714285, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.0, '039490': 0.0,

In [43]:
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                     lookback=lookback + offset, freq='m')
ratio = 0.1
strategy = 'foreign'

sliced_data = factor_data.get_sliced_data(total_lookback)
account_foreign = simulate_factor(
    ohlcv_data=sliced_data['ohlcv_data'],
    market_cap_data=sliced_data['market_cap_data'],
    fundamental_data=None,
    trader_data=sliced_data['trader_data'],
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/Users/jh/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_26788/589796934.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')


2020-01-02
2020-01-03
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-28
2020-01-29
2020-01-30
2020-01-31
Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.0, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.14285714285714285, '006800': 0.0, '007070': 0.0, '008770': 0.14285714285714285, '009150': 0.14285714285714285, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.14285714285714285, '020150': 0.14285714285714285, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.1

In [86]:
ratio = 0.1
strategy = 'multifactor'

sliced_data = factor_data.get_sliced_data(simulation_fromdate)
account_multifactor = simulate_factor(
    ohlcv_data=sliced_data['ohlcv_data'],
    market_cap_data=None,
    fundamental_data=None,
    trader_data=None,
    lookback_period=None,
    skip_period=None,
    strategy_name=strategy,
    buying_ratio=ratio)

/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_27201/1471602628.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')
/var/folders/1j/d_cg9rn93xn390pf9vs669bh0000gn/T/ipykernel_27201/1471602628.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')


2020-01-02
2020-01-03
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-28
2020-01-29
2020-01-30
2020-01-31
Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.1, '001450': 0.0, '001570': 0.0, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.1, '005940': 0.0, '006260': 0.1, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.1, '009150': 0.0, '009830': 0.1, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.1, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.0, '020150': 0.1, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.1, '034730': 0.0, '035250': 0.0, '036460': 0.1, '036570': 0.0, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.0, '05

In [123]:
#시각화
kospi = data_loader.load_index_data(ticker_list=['1001'], freq='m', delay=1)
kospi_returns = kospi.loc[simulation_fromdate:]
kospi_returns = kospi_returns['close'] * (100000000 / kospi_returns['close'][0])
kospi_returns = date_adjust(index_df=ohlcv_data_day, df=kospi_returns)
kospi_returns.name = 'kospi_return'
kospi_returns.index.name = 'date'
def metric_calculate(df: pd.DataFrame, fromdate: str):
    df = df.squeeze().loc[fromdate:]
    df_pct = df.pct_change()
    df_pct.iloc[0] = 0.0
    return f"CAGR: {cagr(returns=df_pct, freq='m')}, " \
           f"MDD: {mdd(returns=df_pct)}, " \
           f"Sharpe Ratio: {sharpe_ratio(returns=df_pct, freq='m')}, " \
           f"Sortino Ratio: {sortino_ratio(returns=df_pct, freq='m')}"
# 총자산 정보 가져오기
total_asset1 = pd.DataFrame(account_relative.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset2 = pd.DataFrame(account_per.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset3 = pd.DataFrame(account_pbr.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset4 = pd.DataFrame(account_dividend.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset5 = pd.DataFrame(account_small.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset6 = pd.DataFrame(account_lowvol.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset7 = pd.DataFrame(account_individual.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset8 = pd.DataFrame(account_institutional.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset9 = pd.DataFrame(account_foreign.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset10 = pd.DataFrame(account_multifactor.account_history)[
    ['date', 'total_asset']].set_index('date')

# 자산 정보 결합하기
total_assets = pd.concat(
    [kospi_returns, total_asset1, total_asset2, total_asset3, total_asset4,
     total_asset5, total_asset6, total_asset7, total_asset8, total_asset9,
     total_asset10
     ],
    axis=1)
total_assets.columns = ["코스피", "상대모멘텀", "PER(가치)", "PBR(가치)", "배당",
                        "소형주", "로우볼", "개인", "기관", "외국인", '멀티팩터'
                        ]

# 자산 변화 시각화하기
color_map = {"멀티팩터": "black"}
fig = px.line(data_frame=total_assets, color_discrete_map=color_map)
fig.show()

KeyError: '지수명'

In [113]:
#성과 지표 계산
metric_calculate(df=kospi_returns, fromdate=simulation_fromdate)

NameError: name 'metric_calculate' is not defined